In [1]:
import numpy as np
import math
import pandas as pd

In [2]:
def get_deltaY_amm(x0,y0,deltaX):
    k=x0 * y0
    return k/(x0+deltaX) - k/x0

def get_deltaX_amm(x0,y0,deltaY):
    k = x0 *y0
    return k/(y0+deltaY) - k/y0

def integ(k,q0,q_new,qi,c):

    if c==1:
        return k/(qi**c) * math.log(q0/q_new)
    else:
        return k/(qi**c)/(c-1) * (q0**(c-1)-q_new**(c-1))

def get_deltaY(x0, y0, i, c, deltaX):
    k=x0 * y0
    xi = (k/i)**0.5
    x_new = x0+deltaX

    #if selling to amm and oracle price is higher i.e. 
    # oracle token balance is lower or vice versa
    if (deltaX>0 and x0 >= xi) or (deltaX<0 and x0 <= xi):
        deltaY = get_deltaY_amm(x0,y0,deltaX)
    elif (deltaX> 0 and x_new <= xi) or (deltaX < 0 and x_new >= xi):
        deltaY = integ(k,x0,x_new,xi,c)
    else:
        deltaY = integ(k,x0,xi,xi,c) + k/x_new-k/xi
        # or 
        # deltaY = integ(k,x0,xi,xi,c) + get_deltaY_amm(xi, yi, x_new - xi)
    
    return deltaY

def get_deltaX(x0, y0, i_, c, deltaY): #here i_ is actually 1/i
    k=x0 * y0
    yi = (k/i_)**0.5
    y_new = y0+deltaY
    
    #if selling to amm and oracle price is higher 
    # i.e. oracle token balance is lower or vice versa
    if (deltaY>0 and y0 >= yi) or (deltaY<0 and y0 <= yi):
        deltaX = get_deltaX_amm(x0,y0,deltaY)
    elif (deltaY> 0 and y_new <= yi) or (deltaY < 0 and y_new >= yi):
        deltaX = integ(k,y0,y_new,yi,c)
    else:
        deltaX = integ(k,y0,yi,yi,c) + k/y_new-k/yi
        # or 
        # deltaX = integ(k,y0,yi,yi,c) + get_deltaX_amm(xi, yi, y_new - yi)
    
    return deltaX

In [3]:
# x0=37
# y0=126
# i=3.0
# c=1.0
# deltaX = 3.0
# deltaY=get_deltaY(x0,y0,i,c,deltaX)

# print("deltaY = ",deltaY, get_deltaY_amm(x0,y0,deltaX))
# print("deltaX = ",get_deltaX(x0,y0,1/i,c,deltaY), get_deltaX_amm(x0,y0,deltaY))

In [4]:
x0=1000
y0=1000
i=0.5
c=0.5
deltaX = 10
deltaY=get_deltaY(x0,y0,i,c,deltaX)

print("deltaY = ",deltaY, get_deltaY_amm(x0,y0,deltaX))
print("deltaX = ",get_deltaX(x0,y0,1/i,c,deltaY), get_deltaX_amm(x0,y0,deltaY))

deltaY =  -8.34641792401218 -9.900990099009846
deltaX =  9.988187440218532 8.41666694385276


In [5]:
c_vals = np.array([i for i in range(0,200,10)]) / 100
bid_ask = np.array([get_deltaY(x0,y0,i,c,deltaX) + get_deltaY(x0,y0,i,c,-deltaX) for c in c_vals])
df = pd.DataFrame()
df['c'] = c_vals
df['bid_ask'] = bid_ask - bid_ask[0]
# df['bid_ask%'] = bid_ask/(deltaX*oracle_price)
print("Bid ask spread for buying 10btc with $36m liquidity over cpmm price with oracle price 1% lower:")
print(df)

Bid ask spread for buying 10btc with $36m liquidity over cpmm price with oracle price 1% lower:
      c   bid_ask
0   0.0  0.000000
1   0.1  0.332512
2   0.2  0.653857
3   0.3  0.964409
4   0.4  1.264530
5   0.5  1.554572
6   0.6  1.834873
7   0.7  2.105759
8   0.8  2.367548
9   0.9  2.620544
10  1.0  2.865044
11  1.1  3.101331
12  1.2  3.329683
13  1.3  3.550366
14  1.4  3.763637
15  1.5  3.969746
16  1.6  4.168932
17  1.7  4.361429
18  1.8  4.547460
19  1.9  4.727244
